# Opis notatnika

### Importy

In [1]:
import sys
from pathlib import Path
import pandas as pd

SRC = (Path("..")).resolve()
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))


In [5]:
from classes.pokemon_team import PokemonTeam
from schemas import PokemonSchema
from visualization.utils import summarize
from data.data import get_pokemons
from report.reports import PdfReport
from experiments.sa_experiments import run_multiple_runs, compare_to_hill_climb, compare_to_random_search
from visualization.plots import visualize_opponents_typing_distribution, visualize_opponents_stat_sums_violin
from constants import REPORT_DIR

In [3]:
from solvers import EvolutionaryAlgorithmPokemonSolver

### Funckje pomocnicze

In [ ]:
import numpy as np
import pandas as pd
from pandera.typing import DataFrame

def ea_run_multiple_runs_df(
    pokemons: DataFrame[PokemonSchema],
    solver: EvolutionaryAlgorithmPokemonSolver,
    runs: int = 8,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Returns:
      df_runs: one row per run with best team info
      df_summary: aggregated stats (mean/median/std/min/max/count)
    """
    rows = []

    for i in range(runs):
        print(f"EA Run {i+1}/{runs}")
        best_team, best_fitness, _, _ = solver.solve(pokemons)

        print(f"Best fitness: {best_fitness}")
        print(f"Best team: {best_team}")

        # WAŻNE ŻEBY REPREZENTACJA DRUŻYNY BYŁA POD NAZWĄ 'pokemons', bo tam df jest pod spodem resizowany żeby w raporcie się zmieściło !!!!! !!!!!
        rows.append(
            {
                "run": i,
                "solver": "EA",
                "elite_size": solver.elite_size,
                "fitness": float(best_fitness),
                "stats_sum": int(best_team.get_stats_sum()),
                "pokemons": repr(best_team),
            }
        )

    df_runs = pd.DataFrame(rows)

    df_summary = (
        df_runs.groupby("solver")["fitness"]
        .agg(["mean", "median", "std", "min", "max", "count"])
        .reset_index()
        .sort_values("mean", ascending=False)
        .round(6)
    )

    return df_runs, df_summary


### Parametry startowe

In [8]:
pokemons = get_pokemons()
all_pokemons = get_pokemons(include_only_final_evolutions=False)
all_pokemons_and_legendaries = get_pokemons(include_legendary=True, include_only_final_evolutions=False)

In [9]:
print("Ilość dostępnych pokemonów bazowo:", len(pokemons))
print("Ilość dostępnych pokemonów (wszystkie ewolucje):", len(all_pokemons))
print("Ilość dostępnych pokemonów (wszystkie ewolucje + legendarne):", len(all_pokemons_and_legendaries))

Ilość dostępnych pokemonów bazowo: 359
Ilość dostępnych pokemonów (wszystkie ewolucje): 731
Ilość dostępnych pokemonów (wszystkie ewolucje + legendarne): 801


In [10]:
solver = EvolutionaryAlgorithmPokemonSolver()

# EA na podstawowych wartościach

In [14]:
ea_runs_df, ea_summary_df = ea_run_multiple_runs_df(
    pokemons=pokemons,
    solver=solver,
    runs=1,
)

report = report = PdfReport(str(REPORT_DIR / "ea_first_test.pdf"), title="Evolutionary Algorithm - First Test")

report.add_text(
    f"""Setup:
     - Population size: {solver.population_size}
     - Generations: {solver.generations}
     - Mutation rate: {solver.mutation_rate}
     - Elite size: {solver.elite_size}
     - Tournament size: {solver.tournament_size}
"""
)

report.add_dataframe(ea_runs_df, title="EA - Results of individual runs")
report.add_dataframe(ea_summary_df, title="EA - Summary statistics")

report.write()

EA Run 1/1
Best fitness: 0.2687833333333333
Best team: PokemonTeam(size=6, names=['Poliwrath', 'Musharna', 'Porygon-Z', 'Haxorus', 'Mamoswine', 'Gogoat'])
